In [2]:
%load_ext autoreload
%autoreload 2
import pickle
import os, sys
root_path = os.path.realpath('../')
sys.path.append(root_path)

import torch
from pathlib import Path
import numpy as np
import random


from utils.data import make_blobs_dataset
from utils.nnet import get_device

from utils.nnet import from_gpu
from utils.eval import compute_accuracy
from hebbcl.logger import MetricLogger
from hebbcl.model import ScaledNet2Hidden, ScaledNet2Hidden2Ctx
from hebbcl.trainer import Optimiser, train_on_blobs
from hebbcl.parameters import parser
from hebbcl.tuner import HPOTuner

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# obtain params
args = parser.parse_args(args=[])

# set checkpoint directory
save_dir = (
        Path("checkpoints") / "test_allhebb"
    ) 

# get device (gpu/cpu)
args.device = get_device(args.cuda)[0]

# override defaults 
# args.n_features = None # todo need to change to flattened size of image
args.n_episodes = 20
args.lrate_sgd = 0.5
args.perform_hebb = False
args.ctx_scaling = 2
args.ctx_avg = False

# create dataset 
dataset = make_blobs_dataset(args)

# instantiate model and optimiser:
model = ScaledNet2Hidden2Ctx(args)
optimiser = Optimiser(args)

# send model to device (GPU?)
model = model.to(args.device)


# train model
# send data to gpu
x_train = torch.from_numpy(dataset["x_train"]).float().to(args.device)
y_train = torch.from_numpy(dataset["y_train"]).float().to(args.device)

# test: create test sets
x_a = torch.from_numpy(dataset["x_task_a"]).float().to(args.device)
r_a = torch.from_numpy(dataset["y_task_a"]).float().to(args.device)

x_b = torch.from_numpy(dataset["x_task_b"]).float().to(args.device)
r_b = torch.from_numpy(dataset["y_task_b"]).float().to(args.device)

x_both = (
    torch.from_numpy(np.concatenate((dataset["x_task_a"], dataset["x_task_b"]), axis=0))
    .float()
    .to(args.device)
)
r_both = (
    torch.from_numpy(np.concatenate((dataset["y_task_a"], dataset["y_task_b"]), axis=0))
    .float()
    .to(args.device)
)

f_both = torch.from_numpy(dataset["f_all"]).float().to(args.device)



# loop over data and apply optimiser
idces = np.arange(len(x_train))
for ii, x, y in zip(idces, x_train, y_train):
    optimiser.step(model, x, y)
    if ii % args.log_interval == 0:            
        if args.verbose:
            print(
                "step {}, loss: task a {:.4f}, task b {:.4f} | acc: task a {:.4f}, task b {:.4f}".format(
                    str(ii),
                    from_gpu(optimiser.loss_funct(r_a, model(x_a))).ravel()[0],
                    from_gpu(optimiser.loss_funct(r_b, model(x_b))).ravel()[0],
                    compute_accuracy(r_a, model(x_a)),
                    compute_accuracy(r_b, model(x_b)),
                )
            )             

print("done")


step 0, loss: task a -0.0003, task b 0.0021 | acc: task a 0.5000, task b 0.5000
step 50, loss: task a -0.1261, task b 0.0173 | acc: task a 0.5000, task b 0.5000
step 100, loss: task a -11.7490, task b 1.3865 | acc: task a 1.0000, task b 0.5000
step 150, loss: task a -14.9377, task b 0.5166 | acc: task a 1.0000, task b 0.5000
step 200, loss: task a -14.9724, task b 0.3387 | acc: task a 1.0000, task b 0.5000
step 250, loss: task a -14.9824, task b 0.3281 | acc: task a 1.0000, task b 0.5000
step 300, loss: task a -14.9874, task b 0.2937 | acc: task a 1.0000, task b 0.5000
step 350, loss: task a -14.9903, task b 0.2793 | acc: task a 1.0000, task b 0.5000
step 400, loss: task a -14.9922, task b 0.2493 | acc: task a 1.0000, task b 0.5000
step 450, loss: task a -14.9934, task b 0.2332 | acc: task a 1.0000, task b 0.5000
step 500, loss: task a -14.9944, task b 0.0589 | acc: task a 1.0000, task b 0.5000
step 550, loss: task a -6.4224, task b -13.6189 | acc: task a 0.7000, task b 0.9500
step 600

## image preprocessing

In [4]:

from PIL import Image
import pickle
import matplotlib.pyplot as plt
from utils.data import resize_images

In [70]:
# files = ["training_data_north_withgarden", "training_data_south_withgarden", "test_data_north_withgarden", "test_data_south_withgarden"]

# for fn in files:
#     resize_images(filename=fn, size=(24,24))

In [95]:
with open("../datasets/test_data_south_withgarden_ds24.pkl","rb") as f:
    data = pickle.load(f)
data["contexts"]
contexts = np.zeros((len(data["images"]),2))
contexts[:,data["contexts"]-1] = 1
np.concatenate((data["images"].astype("float")/255,contexts),axis=1)

array([[0.69803922, 0.69803922, 0.69803922, ..., 0.69803922, 0.        ,
        1.        ],
       [0.69803922, 0.69803922, 0.69803922, ..., 0.69803922, 0.        ,
        1.        ],
       [0.69803922, 0.69803922, 0.69803922, ..., 0.69803922, 0.        ,
        1.        ],
       ...,
       [0.69803922, 0.69803922, 0.69803922, ..., 0.69803922, 0.        ,
        1.        ],
       [0.69803922, 0.69803922, 0.69803922, ..., 0.69803922, 0.        ,
        1.        ],
       [0.69803922, 0.69803922, 0.69803922, ..., 0.69803922, 0.        ,
        1.        ]])